In [1]:
import tensorflow as tf

2025-01-13 15:24:51.441098: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
input_channels, output_channels = 3, 8
W, H = 24, 48
kernel_sizes = [3,3]

In [15]:
input = tf.random.normal([H, W, input_channels])
kernel = tf.random.normal(kernel_sizes+[input_channels, output_channels])
bias = tf.random.normal([output_channels])

In [ ]:
def maual_conv2D(inputs, kernel, bias, stride=1, padding='valid'):
    input_h, input_w, input_channels= inputs.shape
    kernel_h, kernel_w, _, output_channels = kernel.shape
    if padding == 'valid':
        output_h = (input_h - kernel_h) // stride + 1
        output_w = (input_w - kernel_w) // stride + 1
    elif padding == 'same':
        output_h = input_h
        output_w = input_w
        pad_height = ((input_h - 1) * stride + kernel_h -input_h) // 2
        pad_width = ((input_w - 1) * stride + kernel_w - input_w) // 2 
        inputs = tf.pad(inputs, [[pad_height, pad_height],[pad_width, pad_width], [0,0]])
    output = tf.zeros([input_h, input_w, output_channels])

    for i in range(output_h):
        for j in range(output_w):
            for k in range(output_channels):
                input_slice = inputs[i*stride:i*stride+kernel_h, j*stride:j*stride+kernel_w, :]
                # 每个位置、每个通道输出的值是感受野内的输入（kernel_size * kernel_size * input_channels)和kernel的哈达玛积的sum + 对应通道的bias
                # 复杂度为 kernel_size * kernel_size * input_channels
                output = tf.tensor_scatter_nd_add(output, [[i,j,k]], [tf.reduce_sum(tf.multiply(input_slice, kernel[:,:,:,k])) + bias[k]])
    return output
maual_conv2D(inputs=input, kernel=kernel, bias=bias, padding='same')



<tf.Tensor: shape=(48, 24, 8), dtype=float32, numpy=
array([[[ 3.29064131e+00,  4.04826593e+00,  1.71673775e+00, ...,
         -2.31825566e+00,  1.84074986e+00,  1.14289749e+00],
        [ 1.23725758e+01, -9.91231441e-01,  4.81099606e+00, ...,
         -3.34339714e+00,  1.12385607e+01,  8.60099125e+00],
        [ 1.10461988e+01,  1.59300137e+00,  5.64303303e+00, ...,
         -3.35363412e+00,  5.93191481e+00,  5.28215218e+00],
        ...,
        [ 3.04481936e+00,  2.78588367e+00,  3.06177974e+00, ...,
         -3.08427954e+00,  3.38887572e-01,  3.71340156e+00],
        [ 4.29789114e+00,  2.93016052e+00,  5.23214722e+00, ...,
         -6.77662849e+00, -6.30140638e+00, -4.72761011e+00],
        [-7.19560027e-01, -6.79451799e+00, -4.07802916e+00, ...,
         -1.07289639e+01,  1.44882953e+00, -4.47280359e+00]],

       [[ 2.43757391e+00,  4.37989521e+00,  6.89842224e+00, ...,
         -2.80651331e+00,  4.12292671e+00,  2.71082568e+00],
        [ 6.53147697e-03, -3.53776574e+00,  8.3571